# Demo with `tiled.client` (Python) API

Demonstrate, using the `tiled.client` Python API, how to select runs between a set of dates and then by plan name.

In [1]:
import datetime
from tiled.client import from_uri
from tiled.client.cache import Cache
from tiled.utils import tree
import tiled.queries

In [2]:
tiled_server = "localhost"
tiled_server_port = 8000

In [3]:
# connect our client with the server
uri = f"http://{tiled_server}:{tiled_server_port}"
print(f"{uri=}")
client = from_uri(uri, cache=Cache.in_memory(2e9))
print(f"{client=}")

uri='http://localhost:8000'
client=<Node {'training', 'class_2021_03', 'files'}>


In [4]:
# use the class catalog
cat = client["class_2021_03"]
print(f"{cat=}")

cat=<Node {'83cae163-e32b-4e62-887b-66f03f61ec33', ...} ~40 entries>


In [5]:
# define some helper functions which make simplify the searches
def iso2time(isotime):
    return datetime.datetime.timestamp(datetime.datetime.fromisoformat(isotime))

def QueryTimeSince(isotime):
    return tiled.queries.Key("time") >= iso2time(isotime)

def QueryTimeUntil(isotime):
    return tiled.queries.Key("time") < iso2time(isotime)

In [6]:
# Find all runs in the catalog between these two ISO8601 dates.
start_time = "2021-03-17 00:30"
end_time = "2021-05-19 15:15"
cat = cat.search(QueryTimeSince(start_time)).search(QueryTimeUntil(end_time))
print(f"{cat=}")

cat=<Node {'d6e1bad1-92f6-4257-8d5e-53172cd9b784', ...} ~12 entries>


In [7]:
# Find run(s) which match given metadata: given plan_name
plan_name = "rel_scan"
cat = cat.search(tiled.queries.Key("plan_name") == plan_name)
print(f"{cat=}")

cat=<Node {'d6e1bad1-92f6-4257-8d5e-53172cd9b784', ...} ~6 entries>


In [8]:
# With last (latest) run:
run = cat.values()[-1]
print(f"last run: {run=}")

last run: run=<Node {'baseline', 'primary'}>


In [9]:
# Get overall metadata from this run.
print(f"Run metadata: {run.metadata=}")

Run metadata: run.metadata=DictView({'start': {'uid': 'a4c77cc5-dd49-4fc2-90e6-e3cd92667037', 'time': 1621455296.577983, 'login_id': 'prjemian@zap', 'scan_id': 188, 'pid': 3712584, 'objective': 'Demonstrate UB matrix save & restore', 'notebook': 'UB_autosave', 'beamline_id': 'APS_Python_training_2021', 'versions': {'apstools': '1.5.0rc1', 'bluesky': '1.6.7', 'databroker': '1.2.2', 'epics': '3.4.3', 'h5py': '3.2.1', 'intake': '0.6.2', 'matplotlib': '3.3.4', 'numpy': '1.20.1', 'ophyd': '1.6.1', 'pyRestTable': '2020.0.3', 'spec2nexus': '2021.1.8'}, 'instrument_name': 'class_2021_03', 'proposal_id': 'training', 'plan_type': 'generator', 'plan_name': 'rel_scan', 'detectors': ['scaler1'], 'motors': ['zaxis_h'], 'num_points': 8, 'num_intervals': 7, 'plan_args': {'detectors': ["ScalerCH(prefix='gp:scaler1', name='scaler1', read_attrs=['channels', 'channels.chan01', 'channels.chan01.s', 'channels.chan02', 'channels.chan02.s', 'channels.chan03', 'channels.chan03.s', 'channels.chan04', 'channels.

In [10]:
# What are the data streams in this run?
stream_names = run.metadata["summary"]["stream_names"]
print(f"streams: {stream_names=}")

streams: stream_names=['baseline', 'primary']


In [11]:
# Get the data from the data stream named primary (the canonical main data).
if "primary" in stream_names:
    stream_data = run["primary"]
    print(f"{stream_data['data']=}")
    # What is the metadata for this stream?
    print(f"{stream_data.metadata=}")

stream_data['data']=<DatasetClient {'time', 'timebase', 'I0', 'scint', 'diode', ...} ~8 entries>
stream_data.metadata=DictView({'descriptors': [{'run_start': 'a4c77cc5-dd49-4fc2-90e6-e3cd92667037', 'time': 1621455297.2849786, 'data_keys': {'timebase': {'source': 'PV:gp:scaler1.S1', 'dtype': 'number', 'shape': [], 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0, 'precision': 0, 'object_name': 'scaler1'}, 'I0': {'source': 'PV:gp:scaler1.S2', 'dtype': 'number', 'shape': [], 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0, 'precision': 0, 'object_name': 'scaler1'}, 'scint': {'source': 'PV:gp:scaler1.S3', 'dtype': 'number', 'shape': [], 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0, 'precision': 0, 'object_name': 'scaler1'}, 'diode': {'source': 'PV:gp:scaler1.S4', 'dtype': 'number', 'shape': [], 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0, 'precision': 0, 'object_name': 'scaler1'}, 'scaler1_time': {'source': 'PV:gp:scaler1.T', 'd

----

Make a single function that searches by date range and metadata keys.

In [12]:
def get_runs(cat, since=None, until=None, **keys):
    """
    Return a new catalog, filtered by search terms.
    
    Runs will be selected with start time `>=since` and `< until`.
    If either is `None`, then the corresponding filter will not be
    applied.
    
    Parameters
    
    `cat` obj :
        This is the catalog to be searched.
        `Node` object returned by tiled.client.
    `since` str :
        Earliest start date (& time), in ISO8601 format.
    `until` str :
        Latest start date (& time), in ISO8601 format.
    `keys` dict :
        Dictionary of metadata keys and values to be matched.
    """
    if since is not None:
        cat = cat.search(QueryTimeSince(since))
    if until is not None:
        cat = cat.search(QueryTimeUntil(until))
    
    for k, v in keys.items():
        cat = cat.search(tiled.queries.Key(k) == v)
    return cat

In [13]:
get_runs(client["class_2021_03"], since="2021-03-17 00:30")

<Node {'d6e1bad1-92f6-4257-8d5e-53172cd9b784', ...} ~16 entries>

In [14]:
get_runs(client["class_2021_03"], until="2021-05-19 15:15")

<Node {'83cae163-e32b-4e62-887b-66f03f61ec33', ...} ~36 entries>

In [15]:
get_runs(client["class_2021_03"], since="2021-03-17 00:30", until="2021-05-19 15:15")

<Node {'d6e1bad1-92f6-4257-8d5e-53172cd9b784', ...} ~12 entries>

In [16]:
get_runs(client["class_2021_03"], since="2021-03-17 00:30", until="2021-05-19 15:15", plan_name="rel_scan")

<Node {'d6e1bad1-92f6-4257-8d5e-53172cd9b784', ...} ~6 entries>

Compare this last search with the results above.  Both times, 6 entries were found.

In [17]:
# One last search, for a specific `scan_id`:
get_runs(client["class_2021_03"], scan_id=188)

<Node {'a4c77cc5-dd49-4fc2-90e6-e3cd92667037'}>